In [68]:
import sklearn
import pandas
import seaborn
import numpy as np

In [69]:
dataset = pandas.read_csv('cs.csv')

# dataset["round_winner"].replace(to_replace ="CT", value = False, inplace=True)
# dataset["round_winner"].replace(to_replace ="T", value = True, inplace=True)

dataset = pandas.get_dummies(dataset, columns=["map"])


In [70]:
X = dataset.loc[:, dataset.columns != 'round_winner'] 
y = np.array(dataset.loc[:, dataset.columns == 'round_winner']).ravel()

In [71]:
# importa o método do sklearn que separa em conjunto de treino e teste
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

In [72]:
# importa a arvore de decisão
from sklearn.tree import DecisionTreeClassifier
# instancia o classificador 
clf = DecisionTreeClassifier(criterion='gini', random_state=42)
# treina o classificador com o conjunto de treino
clf.fit(X_train, y_train)
# prediz as classes do conjunto de teste
predict = clf.predict(X_test)

In [73]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, predict)
print(f'A acurácia do Classificador é: {acc :.2%}')

A acurácia do Classificador é: 82.27%
